In [2]:
#套件
from __future__ import print_function, division
import pandas as pd
import numpy as np
from sklearn import preprocessing
from pathlib import Path
from torchvision import datasets,transforms
import torch.optim as optim
import torch.nn
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from time import sleep
from tqdm import tqdm, trange
from PIL import Image
from torch.autograd import Variable
import torch.nn.functional as FUN
from scipy import io
import torchvision.transforms as T
import pickle
import torchvision.datasets as dsets
from scipy.misc import imsave
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.utils.rnn as rnn_utils
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
import os

In [3]:
#匯入訓練資料

class_number = 100

'''匯入所有label'''
train_tensor_ = []
for t in range(class_number):
    with open('pickle\cifar100_test/class'+str(t)+'/cifar100_class'+str(t)+'_tensor.pickle', 'rb') as f:
        train_tensor_.append(pickle.load(f))


# val_label_ = []
# for t in range(class_number):
#     with open('pickle\cifar_fcn/class'+str(t)+'/cifar_val_class'+str(t)+'_label.pickle', 'rb') as f:
#         val_label_.append(pickle.load(f))


# '''匯入所有confidence'''
# train_tensor_ = []
# for t in range(class_number):
#     with open('pickle\cifar_fcn/class'+str(t)+'/cifar_class'+str(t)+'_tensor.pickle', 'rb') as f:
#         train_tensor_.append(pickle.load(f))


# val_tensor_ = []
# for t in range(class_number):
#     with open('pickle\cifar_fcn/class'+str(t)+'/cifar_val_class'+str(t)+'_tensor.pickle', 'rb') as f:
#         val_tensor_.append(pickle.load(f))

# print(train_tensor_[1][1])

In [ ]:
#觀察資料

for t in range(len(train_tensor_)):
    print( 'class ' +str(t)+' : '+ str(len(train_tensor_[t])))

In [4]:
#實驗temperature scaling

with open('pickle/cifar100_all/cifar100_all_tensor.pickle', 'rb') as f:  
    result = pickle.load(f)

with open('pickle\cifar100_all/cifar100_all_class_label.pickle', 'rb') as f:
    class_label = pickle.load(f)

with open('pickle\cifar100_all/cifar100_all_path.pickle', 'rb') as f:
    path = pickle.load(f)

In [3]:
#實驗temperature scaling

class calibration_Model(torch.nn.Module):
    def __init__(self):
        # ...
        super(calibration_Model,self).__init__()
        self.temperature = torch.nn.Parameter(torch.rand(1).to(device))

    def forward(self, x):
        # ...
        logits = x
        global_para = self.temperature
        return logits / self.temperature

#參數設置
batch_size = 64
epoch = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#依batch包裝training set
count = 0
batch_package = []
all_data = []
for number in range(len(result)):
    count+=1
    item = result[number].squeeze(0)
    batch_package.append(item)
    if count%batch_size==0:
        a = torch.stack(batch_package)
        all_data.append(a)
        batch_package = []
#依batch包裝label
count = 0
batch_package = []
all_label = []
for number in range(len(class_label)):
    count+=1
    item = class_label[number].squeeze(0)
    batch_package.append(item)
    if count%batch_size==0:
        a = torch.stack(batch_package)
        all_label.append(a)
        batch_package = []


#開始訓練參數

my_model = calibration_Model().to(device)
my_model.__init__()
optimizer = torch.optim.SGD((my_model.parameters()), lr = 0.001)
criterion = torch.nn.CrossEntropyLoss()


for runtime in range(epoch):
    print('epoch : ' + str(runtime) + ' is begining')
    for number in range(len(all_data)):
        bucket = [[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]  #[ 資料個數 , confidence加總 , 分類正確的個數 ]
        new_all_data = []
        for t in range(len(all_data[number])):
            
            optimizer.zero_grad()
            og_soft = FUN.softmax(Variable(all_data[number][t])).data
            new_soft = my_model(og_soft.to(device))
            new_all_data.append(new_soft)
            

        new_data = torch.stack(new_all_data)
        loss = criterion(new_data.to(device),all_label[number].to(device))
        loss.requires_grad_()
        loss.requires_grad
        loss.backward()
        optimizer.step()
        
        #進行validation

        if number % 30 == 0:

            print('grad : ',end='')
            print(loss.grad)

            for pa in my_model.parameters():
                print('parameter : ', end='')
                print(pa.data)

            val_count = 0
            
            all_ece = torch.tensor(0)
            for val_number in range(len(result)):
                val_og_soft = FUN.softmax(Variable(result[val_number].squeeze(0))).data
                val_new_soft = my_model(val_og_soft.to(device))
                val_max_class = torch.argmax(val_new_soft)
                if val_max_class == class_label[val_number]:
                    val_count += 1

                #計算ece
                max = torch.max(val_new_soft)
                max_class = torch.argmax(val_new_soft)
                if 1.0>= max >= 0.8 :
                    bucket[4][0]+=1
                    bucket[4][1] = bucket[4][1]+max
                    if max_class == all_label[number][t]:
                        bucket[4][2]+=1

                if 0.8 > max >= 0.6 :
                    bucket[3][0]+=1
                    bucket[3][1] = bucket[3][1]+max
                    if max_class == all_label[number][t]:
                        bucket[3][2]+=1

                if 0.6 > max >= 0.4 :
                    bucket[2][0]+=1
                    bucket[2][1] = bucket[2][1]+max
                    if max_class == all_label[number][t]:
                        bucket[2][2]+=1

                if 0.4 > max >= 0.2 :
                    bucket[1][0]+=1
                    bucket[1][1] = bucket[1][1]+max
                    if max_class == all_label[number][t]:
                        bucket[1][2]+=1

                if 0.2 > max >= 0.0 :
                    bucket[0][0]+=1
                    bucket[0][1] = bucket[0][1]+max
                    if max_class == all_label[number][t]:
                        bucket[0][2]+=1

                ece= torch.tensor(0)
                for bucket_number in range(len(bucket)): # bucket個數
                    register = 0
                    if bucket[bucket_number][0] ==0 :
                        pass
                    else:
                        con_avg = torch.tensor(bucket[bucket_number][1]) / torch.tensor(bucket[bucket_number][0])
                        acc_avg = torch.tensor(bucket[bucket_number][2]) / torch.tensor(bucket[bucket_number][0])
                        register = (abs(acc_avg-con_avg)) * torch.tensor(bucket[bucket_number][0])
                    ece = ece + register
                ece = ece / torch.tensor(len(result))
                all_ece = all_ece + ece



            print('ece : ',end = '')
            print(all_ece/len(result))
            # print('acc : ',end = '')
            # print(val_count/len(result))



            


    



epoch : 0 is begining
grad : None
parameter : tensor([0.6170], device='cuda:0')


c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\torch\_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:417.)
  return self._grad
c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:89: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
c:\Users\

ece : tensor(0.1696, device='cuda:0')
grad : None
parameter : tensor([0.5982], device='cuda:0')
ece : tensor(0.1777, device='cuda:0')
grad : None
parameter : tensor([0.5862], device='cuda:0')
ece : tensor(0.1724, device='cuda:0')
grad : None
parameter : tensor([0.5672], device='cuda:0')
ece : tensor(0.1771, device='cuda:0')
grad : None
parameter : tensor([0.5451], device='cuda:0')
ece : tensor(0.1781, device='cuda:0')
grad : None
parameter : tensor([0.5128], device='cuda:0')
ece : tensor(0.1840, device='cuda:0')
grad : None
parameter : tensor([0.4926], device='cuda:0')
ece : tensor(0.1833, device='cuda:0')
grad : None
parameter : tensor([0.4754], device='cuda:0')
ece : tensor(0.1806, device='cuda:0')
grad : None
parameter : tensor([0.4435], device='cuda:0')
ece : tensor(0.1851, device='cuda:0')
grad : None
parameter : tensor([0.4219], device='cuda:0')
ece : tensor(0.1874, device='cuda:0')
grad : None
parameter : tensor([0.3902], device='cuda:0')
ece : tensor(0.1852, device='cuda:0')
gr

In [39]:
caliration = result[1300].squeeze(0)/0.4
soft = FUN.softmax(Variable(result[1300].squeeze(0))).data
caliration = FUN.softmax(Variable(caliration))
soft = sorted(soft,reverse=True)
caliration = sorted(caliration,reverse=True)

for t in range(len(caliration)):
    print(soft[t],'-----------',caliration[t])


c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


tensor(0.2547, device='cuda:0') ----------- tensor(0.6595, device='cuda:0')
tensor(0.1888, device='cuda:0') ----------- tensor(0.3121, device='cuda:0')
tensor(0.0603, device='cuda:0') ----------- tensor(0.0180, device='cuda:0')
tensor(0.0306, device='cuda:0') ----------- tensor(0.0033, device='cuda:0')
tensor(0.0237, device='cuda:0') ----------- tensor(0.0017, device='cuda:0')
tensor(0.0168, device='cuda:0') ----------- tensor(0.0007, device='cuda:0')
tensor(0.0139, device='cuda:0') ----------- tensor(0.0005, device='cuda:0')
tensor(0.0135, device='cuda:0') ----------- tensor(0.0004, device='cuda:0')
tensor(0.0114, device='cuda:0') ----------- tensor(0.0003, device='cuda:0')
tensor(0.0114, device='cuda:0') ----------- tensor(0.0003, device='cuda:0')
tensor(0.0106, device='cuda:0') ----------- tensor(0.0002, device='cuda:0')
tensor(0.0099, device='cuda:0') ----------- tensor(0.0002, device='cuda:0')
tensor(0.0086, device='cuda:0') ----------- tensor(0.0001, device='cuda:0')
tensor(0.008